**Using only players considered in mvp voting**

In [9]:
import pandas as pd

In [10]:
stats = pd.read_csv("player_mvp_stats.csv")
years = list(range(1991, 2024))

In [11]:
del stats["Unnamed: 0"]

In [12]:
pd.isnull(stats).sum()
stats_all_players = stats.fillna(0)
stats_all_players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,PF,27.0,LAL,82.0,21.0,26.4,3.1,6.6,0.476,0.1,0.7,0.200,3.0,5.9,0.507,0.486,2.7,3.7,0.738,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,1991,13.8,0.556,0.101,0.557,11.3,16.2,13.8,4.5,1.4,0.6,12.8,15.6,3.2,2.9,6.1,0.135,0.1,-0.2,0.0,1.1,0.0,0.0,0.0,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
1,Byron Scott,SG,29.0,LAL,82.0,82.0,32.1,6.1,12.8,0.477,0.9,2.7,0.324,5.2,10.1,0.517,0.510,1.4,1.8,0.797,0.7,2.3,3.0,2.2,1.2,0.3,1.0,1.8,14.5,1991,14.2,0.534,0.208,0.141,2.5,8.1,5.4,10.3,1.8,0.5,7.1,19.9,3.6,2.9,6.6,0.120,0.3,0.6,0.9,2.0,0.0,0.0,0.0,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
2,Elden Campbell,PF,22.0,LAL,52.0,0.0,7.3,1.1,2.4,0.455,0.0,0.0,0.000,1.1,2.4,0.455,0.455,0.6,0.9,0.653,0.8,1.1,1.8,0.2,0.2,0.7,0.3,1.4,2.8,1991,14.5,0.498,0.000,0.398,12.8,16.4,14.7,3.8,1.5,5.9,10.0,18.4,0.4,0.7,1.1,0.135,-3.0,1.6,-1.4,0.1,0.0,0.0,0.0,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
3,Irving Thomas,PF,25.0,LAL,26.0,0.0,4.2,0.7,1.9,0.340,0.0,0.0,0.000,0.7,1.9,0.340,0.340,0.5,0.8,0.571,0.5,0.7,1.2,0.4,0.2,0.0,0.5,0.9,1.8,1991,7.3,0.388,0.000,0.420,15.7,17.5,16.7,13.5,1.9,0.5,18.0,29.1,-0.3,0.2,-0.1,-0.057,-8.7,-1.6,-10.3,-0.2,0.0,0.0,0.0,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
4,James Worthy,SF,29.0,LAL,78.0,74.0,38.6,9.2,18.7,0.492,0.3,1.2,0.289,8.8,17.5,0.505,0.501,2.7,3.4,0.797,1.4,3.2,4.6,3.5,1.3,0.4,1.6,1.5,21.4,1991,18.1,0.531,0.062,0.183,4.3,9.2,6.9,15.1,1.8,0.7,7.5,24.6,5.5,3.5,9.0,0.143,2.2,0.3,2.5,3.4,0.0,0.0,0.0,Los Angeles Lakers,58.0,24.0,0.707,5.0,106.3,99.6,6.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15233,Terrence Jones,PF,25.0,MIL,54.0,12.0,23.5,4.3,9.1,0.470,0.4,1.4,0.253,3.9,7.7,0.508,0.489,1.9,3.1,0.606,1.2,4.5,5.7,1.1,0.7,1.0,0.9,1.2,10.8,2017,16.1,0.515,0.152,0.346,5.4,20.2,12.7,7.5,1.5,3.5,7.8,21.4,0.4,1.9,2.3,0.086,-1.0,0.4,-0.6,0.5,0.0,0.0,0.0,Milwaukee Bucks,42.0,40.0,0.512,9.0,103.6,103.8,-0.45
15234,Thon Maker,C,19.0,MIL,57.0,34.0,9.9,1.5,3.2,0.459,0.5,1.3,0.378,1.0,1.9,0.514,0.536,0.6,0.9,0.653,0.7,1.3,2.0,0.4,0.2,0.5,0.3,1.5,4.0,2017,14.0,0.558,0.409,0.271,8.4,15.2,11.8,6.2,0.9,4.1,7.7,17.8,0.8,0.5,1.3,0.113,-1.1,-0.9,-2.0,0.0,0.0,0.0,0.0,Milwaukee Bucks,42.0,40.0,0.512,9.0,103.6,103.8,-0.45
15235,Tony Snell,SG,25.0,MIL,80.0,80.0,29.2,3.1,6.8,0.455,1.8,4.4,0.406,1.3,2.3,0.548,0.588,0.6,0.7,0.810,0.3,2.8,3.1,1.2,0.7,0.2,0.7,1.6,8.5,2017,9.7,0.603,0.656,0.107,1.1,11.1,6.2,5.9,1.2,0.5,8.8,12.1,2.1,1.3,3.4,0.071,-1.0,-0.7,-1.7,0.2,0.0,0.0,0.0,Milwaukee Bucks,42.0,40.0,0.512,9.0,103.6,103.8,-0.45
15236,Alperen Sengun,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022,16.0,0.552,0.223,0.442,10.1,18.9,14.5,19.1,1.9,4.1,18.8,22.0,0.8,1.3,2.1,0.068,-1.6,0.6,-1.0,0.4,0.0,0.0,0.0,0,0.0,0.0,0.000,0.0,0.0,0.0,0.00


In [13]:
mvp_players = stats_all_players[stats_all_players["Share"] > 0]
mvp_players.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB',
       'PS/G', 'PA/G', 'SRS'],
      dtype='object')

In [35]:
from sklearn.linear_model import LinearRegression

predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']
    
#Testing on selected year. Training model on data from previous years
train = mvp_players[mvp_players['Year'] < 2023]
test = mvp_players[mvp_players['Year'] == 2023]

X_train = train[predictors]
y_train = train["Share"]

X_test = test[predictors]
y_test = test["Share"]

linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

prediction_votings = linear_model.predict(X_test)

,Predictions
214,0.494943
306,0.099204
736,0.479351
1322,0.103334
2898,0.170877
4272,-0.181065
4274,0.048098
6686,0.046833
10869,0.017990
12647,0.062923


**Modeling**

**Defining a Train/Test Split with training on data before year and testing on current year (modeling what we are going to be doing)**

In [36]:
def train_test_split(year, df):
    predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS','PER','TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP']
    
    #Testing on selected year. Training model on data from previous years
    train = df[df['Year'] < year]
    test = df[df['Year'] == year]
    
    X_train = train[predictors]
    y_train = train["Share"]

    X_test = test[predictors]
    
    return X_train, y_train, X_test, test

**Function to run multiple models**

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor

In [44]:
def run_model(regressor, X_train, y_train, X_test, test):
    model = regressor
    model.fit(X_train, y_train) 
    predictions = model.predict(X_test)
    
    predictions = pd.DataFrame(predictions, columns=["Predictions"], index = test.index)
    combinations = pd.concat([test[["Player", "Share"]], predictions], axis = 1)
    combinations.sort_values("Predictions", ascending = False)
    combinations = combinations.sort_values("Share", ascending = False)
    combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
    combinations = combinations.sort_values("Predictions", ascending = False)
    combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

    mse = mean_squared_error(combinations["Share"], combinations["Predictions"])
    return combinations,mse

**Finding average error metric for model across multiple years to help with model selection**

In [64]:
def backtest(model, years):
    mse_lst = []
    for year in years[5:]:
        X_train, y_train, X_test, test = train_test_split(year, mvp_players)
        results, mse = run_model(model, X_train=X_train, y_train=y_train, X_test=X_test, test=test)
        mse_lst.append(mse)

    return sum(mse_lst)/len(mse_lst)

In [65]:
X_train, y_train, X_test, test = train_test_split(2023, mvp_players)

**Linear Regression Model**

In [66]:
mvp_linear_race, mse = run_model(LinearRegression(), X_train=X_train, y_train=y_train, X_test=X_test, test=test)
mvp_linear_race

,Player,Share,Predictions,Rank,Predicted_Rank
214,Giannis Antetokounmpo,0.606,0.494943,3,1
736,Nikola Jokić,0.674,0.479351,2,2
14849,Joel Embiid,0.915,0.404954,1,3
2898,Jayson Tatum,0.280,0.170877,4,4
1322,Shai Gilgeous-Alexander,0.046,0.103334,5,5
306,Luka Dončić,0.010,0.099204,8,6
12647,Ja Morant,0.001,0.062923,12,7
4274,Domantas Sabonis,0.027,0.048098,7,8
6686,Stephen Curry,0.005,0.046833,9,9
10869,Jimmy Butler,0.003,0.017990,10,10


In [68]:
avg_linear_mse = backtest(LinearRegression(), years=years)
avg_linear_mse

0.055889220717971276

**Ridge Regression Model**

In [69]:
mvp_ridge_race, mse = run_model(Ridge(), X_train=X_train, y_train=y_train, X_test=X_test, test=test)
mvp_ridge_race

,Player,Share,Predictions,Rank,Predicted_Rank
736,Nikola Jokić,0.674,0.532031,2,1
214,Giannis Antetokounmpo,0.606,0.431766,3,2
14849,Joel Embiid,0.915,0.300434,1,3
2898,Jayson Tatum,0.280,0.191334,4,4
306,Luka Dončić,0.010,0.115809,8,5
4274,Domantas Sabonis,0.027,0.101717,7,6
12647,Ja Morant,0.001,0.092814,12,7
6686,Stephen Curry,0.005,0.064230,9,8
10869,Jimmy Butler,0.003,0.048001,10,9
1322,Shai Gilgeous-Alexander,0.046,0.015425,5,10


In [70]:
avg_ridge_mse = backtest(Ridge(), years=years)
print(avg_ridge_mse)

0.039647664949490034


**Random Forest Regressor Model**

In [71]:
mvp_race_rf, mse = run_model(RandomForestRegressor(), X_train=X_train, y_train=y_train, X_test=X_test, test=test)
mvp_race_rf

,Player,Share,Predictions,Rank,Predicted_Rank
736,Nikola Jokić,0.674,0.60108,2,1
14849,Joel Embiid,0.915,0.50125,1,2
214,Giannis Antetokounmpo,0.606,0.40359,3,3
306,Luka Dončić,0.010,0.34517,8,4
10869,Jimmy Butler,0.003,0.26087,10,5
2898,Jayson Tatum,0.280,0.22095,4,6
1322,Shai Gilgeous-Alexander,0.046,0.17701,5,7
6686,Stephen Curry,0.005,0.12509,9,8
4274,Domantas Sabonis,0.027,0.11577,7,9
13682,Donovan Mitchell,0.030,0.08526,6,10


In [72]:
avg_rf_mse = backtest(model=RandomForestRegressor(), years=years)
print(avg_rf_mse)

0.03612851313537888


**Building a XGBoost Model**

In [73]:
mvp_race_XGBoost, mse = run_model(XGBRegressor(), X_train=X_train, y_train=y_train, X_test=X_test, test=test)
mvp_race_XGBoost

,Player,Share,Predictions,Rank,Predicted_Rank
214,Giannis Antetokounmpo,0.606,0.604896,3,1
736,Nikola Jokić,0.674,0.596234,2,2
2898,Jayson Tatum,0.280,0.550493,4,3
14849,Joel Embiid,0.915,0.350196,1,4
306,Luka Dončić,0.010,0.249066,8,5
10869,Jimmy Butler,0.003,0.113087,10,6
1322,Shai Gilgeous-Alexander,0.046,0.107904,5,7
13682,Donovan Mitchell,0.030,0.098837,6,8
4274,Domantas Sabonis,0.027,0.062199,7,9
12647,Ja Morant,0.001,0.061643,12,10


In [74]:
avg_XGBoost_mse = backtest(model=XGBRegressor(), years=years)
avg_XGBoost_mse

0.04087379905890938

**Hyperparameter Tuning a Random Forest Regressor (Best performing model so far)**

In [75]:
from sklearn.model_selection import GridSearchCV

In [82]:
param_grid = {
        'bootstrap':[True],

        'max_depth':[10,20,30,40,50,60,70,80,90,100,None],

        'max_features': ['auto', 'sqrt'],

        'min_samples_leaf': [1, 2, 4],

        'min_samples_split': [2, 5, 10],

        'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
    }

In [83]:
model = RandomForestRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train) 

print(grid_search.best_params_)

Fitting 3 folds for each of 1980 candidates, totalling 5940 fits
{'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 400}


In [78]:
mvp_race_rf_optimized, mse = run_model(RandomForestRegressor(bootstrap=True, max_depth=10, max_features = 'sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators = 200), X_train=X_train, y_train=y_train, X_test=X_test, test=test)
mvp_race_rf_optimized

,Player,Share,Predictions,Rank,Predicted_Rank
736,Nikola Jokić,0.674,0.671965,2,1
14849,Joel Embiid,0.915,0.502355,1,2
214,Giannis Antetokounmpo,0.606,0.412012,3,3
306,Luka Dončić,0.010,0.337429,8,4
2898,Jayson Tatum,0.280,0.234890,4,5
10869,Jimmy Butler,0.003,0.226233,10,6
1322,Shai Gilgeous-Alexander,0.046,0.193062,5,7
4274,Domantas Sabonis,0.027,0.151329,7,8
6686,Stephen Curry,0.005,0.150639,9,9
12647,Ja Morant,0.001,0.099244,12,10


In [79]:
mse

0.034102263025309304

In [81]:
backtest(RandomForestRegressor(bootstrap=True, max_depth=10, max_features = 'sqrt', min_samples_leaf=1, min_samples_split=2, n_estimators = 200), years=years)

0.03357146172194143